In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp

# Append the analysis folder so that you can import custom packages
import sys
sys.path.append(r'C:\Users\Leonardo\Documents\MATLAB\PNN_wholeBrain\analysis')

# Custom packages
import dataIO
import AbaTool
import GraphicTool as gt

# Instantiate an Atlas object from the AbaTool.py file 
# The first time you run this it will download the structures.json file from the Allen Institute server
structuresFile = r"C:\Users\Leonardo\Documents\MATLAB\PNN_wholeBrain\structures.json"
A = AbaTool.Atlas(nodes=structuresFile)
DFM = AbaTool.AnatomyDataFrameManager(A)

# 1. Load all data

Loads data for **diffuse fluorescence** and **dots** for a **single channel** and 
creates a multi-index dataframe (df) containing raw measurements for all areas
of the brain at high resolution

In [ ]:
# --------------------------------------------------------------------
searchPath = r'D:\PizzorussoLAB\proj_PNN-highFatDiet\RESULTS\allData'
channelName = 'pv'     # 'wfa' or 'pv'
# --------------------------------------------------------------------

df = dataIO.allMiceRegions(searchPath=searchPath, channelName=channelName, normCellIntens=True)
df = DFM.multiIndexDf_from_fineDf(df, verbose=True)

# 2. Mid ontology heatmaps

Aggregate data at mid ontology level

In [ ]:
# Dataframe at mid-ontology
midDf = DFM.regionsDf_to_mid(df, verbose=True, normalize=True)
# Select ONLY control animals (CC)
midDf = midDf.xs('CTR', axis=1, level='treat')

midDf.head()

## 2.1 Heatmap - PV Diffuse Fluorescence

In [ ]:
data = midDf.xs('diffuseFluo', axis=1, level='params')

# Create the heatmap
axObj = gt.midOntologyHeatmap(data, A, cmap='PuBu', vmin=0, vmax=2, title='PV Diffuse Fluorescence')

# Save the image if necessary
# plt.savefig("pv_diffuseHeatmap.svg", bbox_inches="tight")

## 2.2 Heatmap - PV Energy

In [ ]:
data = midDf.xs('energy', axis=1, level='params')

# Create the heatmap
axObj = gt.midOntologyHeatmap(data, A, cmap='OrRd', vmin=0, vmax=3, title='PV Energy')

# Save the image if necessary
# plt.savefig("pv_energyHeatmap.svg", bbox_inches="tight")

## 2.3 Heatmap - PV cells Density

In [ ]:
data = midDf.xs('density', axis=1, level='params')

# Create the heatmap
axObj = gt.midOntologyHeatmap(data, A, cmap='BuGn', vmin=0, vmax=200, title='PNN Density')

# Save the image if necessary
# plt.savefig("pv_densityHeatmap.svg", bbox_inches="tight")

## 2.4 Heatmap - PV cells Intensity

In [ ]:
data = midDf.xs('intensity', axis=1, level='params')

# Create the heatmap
axObj = gt.midOntologyHeatmap(data, A, cmap='Purples', vmin=0, vmax=1, title='PNN Intensity')

# Save the image if necessary
# plt.savefig("pv_intensityHeatmap.svg", bbox_inches="tight")

# 3. Coarse ontology bar plots
Aggregate data at coarse ontology level

In [ ]:
# Dataframe at coarse ontology
coarseDf = DFM.regionsDf_to_coarse(df, verbose=True, normalize=True)

# Select only control animals
coarseDf = coarseDf.xs('CTR', axis=1, level='treat')

coarseDf.head()

## 3.1 Barplot - PV Diffuse Fluorescence

In [ ]:
data = coarseDf.xs('diffuseFluo', axis=1, level='params')

# Plot the data
g = gt.coarseOntologyBarplot(data, A, 
    title='PV Diffuse Fluorescence',
    cmap='PuBu',
    xlabel='(A.U.)',
    areaNames=True,
    )

# Display the statistics
areaNames = A.ids_to_names(data.index.tolist())
stat, pval = ttest_1samp(data, 1, axis=1, nan_policy='omit')
for i,area in enumerate(areaNames):
    print(f'{area:25}\t t: {stat[i]:.3f}\t p:{pval[i]:.4f}')

# plt.savefig('pv_coarseDiffFluo.svg',bbox_inches="tight")

## 3.2 Barplot - PV Energy

In [ ]:
data = coarseDf.xs('energy', axis=1, level='params')

# Plot the data
g = gt.coarseOntologyBarplot(data, A, 
    title='PV Energy',
    cmap='OrRd',
    xlabel='(A.U.)',
    areaNames=True,
    )

# Display the statistics
areaNames = A.ids_to_names(data.index.tolist())
stat, pval = ttest_1samp(data, 1, axis=1, nan_policy='omit')
for i,area in enumerate(areaNames):
    print(f'{area:25}\t t: {stat[i]:.3f}\t p:{pval[i]:.4f}')

# plt.savefig('pv_coarseEnergy.svg',bbox_inches="tight")

# 4. Interactions
Explore the relationship between **PV Diffuse Fluorecence** and **PV Energy**

## 4.1 Mid-ontology

In [ ]:
data = midDf.groupby('params', axis=1).mean()

### Plot brain regions for a single major subdivision

In [ ]:
areaName = 'Isocortex'
toPlot = data.loc[A.names_to_ids([areaName])[0]]

ax = gt.metricsCorrelation(toPlot, A, 
    x='diffuseFluo',
    y='energy',
    txtLoc = 'br',
    xlabel = 'PV Diffuse Fluorescence (A.U.)',
    ylabel = 'PV Energy (A.U.)',
    title = areaName
    )

### Plot all 12 major subdivisions

In [ ]:
f, axs = plt.subplots(nrows=3, ncols=4, figsize=(16,10), squeeze=True)

for i, ax in enumerate(f.axes):
    thisRegion = A.get_major_divisions_ids()[i]
    toPlot = data.loc[thisRegion]

    gt.metricsCorrelation(toPlot, A,
        ax = ax,
        x='diffuseFluo',
        y='energy',
        txtLoc = 'tl' if i in [1,2,3,4] else 'br',
        xlabel = 'PV Diffuse Fluorescence (A.U.)' if i==8 else None,
        ylabel = 'PV Energy (A.U.)' if i==8 else None,
        title = A.ids_to_names([thisRegion])[0],
        fontScaling = 0.9
    )

# plt.savefig('pv_allMidAreasCorrelation.svg',bbox_inches="tight")

## 4.1 Coarse ontology

In [ ]:
# Aggregate data at coarse resolution
data = coarseDf.groupby('params', axis=1).mean()

# Calculate SEM for Diffuse Fluorescence and Energy for displaying errorbars
errors = coarseDf.groupby('params', axis=1).sem()
data['errDiffuse'] = errors['diffuseFluo']
data['errEnergy'] = errors['energy']

### Plot coarse areas with errorbars

In [ ]:
f, ax = plt.subplots(figsize=(6,5))

_ = gt.metricsWithErrors(data, A,
    ax=ax,
    x='diffuseFluo',
    y='energy',
    err_x='errDiffuse',
    err_y='errEnergy',
    xlabel='PV Diffuse Fluorescence (A.U.)',
    ylabel='PV Energy (A.U.)',
    annotations=True,
    )

plt.savefig('allCoarseAreasCorrelation.svg',bbox_inches="tight")

# Interactive plot
Creates an interactive plot for a single major subdivision where you can check which dot corresponds to which area.  
It's useful to create arrows with annotations for particular brain regions on the correlation plots.

In [ ]:
import plotly.express as px

regionName = 'Isocortex'
thisRegion = A.names_to_ids([regionName])[0]
toPlot = data.xs(thisRegion, axis=0, level=0).copy()
toPlot['name'] = A.ids_to_names(toPlot.index.tolist())
toPlot['acro'] = A.ids_to_acronyms(toPlot.index.tolist())


fig = px.scatter(toPlot, x="diffuseFluo", y="energy", 
    title=A.ids_to_names([thisRegion])[0],
    hover_data=['acro'])


fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.show()



# TO REMOVE - Diet Effects

In [ ]:
# --------------------------------------------------------------------
searchPath = r'D:\PizzorussoLAB\proj_PNN-highFatDiet\RESULTS\allData'
channelName = 'wfa'     # 'wfa' or 'pv'
# --------------------------------------------------------------------

df = dataIO.allMiceRegions(searchPath=searchPath, channelName=channelName, normCellIntens=True)
df = DFM.multiIndexDf_from_fineDf(df, verbose=True)

# Dataframe at mid-ontology
midDf = DFM.regionsDf_to_mid(df, verbose=True, normalize=True)
midDf

In [ ]:
from scipy.stats import ttest_ind
from statsmodels.stats import multitest

variable = 'diffuseFluo'

idx = pd.IndexSlice
# Calculate p-values for each row
stat = []
for area, series in midDf.iterrows():
    controls = series.loc[idx['CTR',:,variable]]
    hfd = series.loc[idx['HFD',:,variable]]
    
    if (np.sum(np.isnan(controls.values)) > 3) or (np.sum(np.isnan(hfd.values)) > 3):
        result = [100, 1]
    else:
        # Perform the t-test
        result = ttest_ind(controls,hfd,nan_policy='omit')
    # Save the results
    temp = {'area':A.ids_to_names([area[1]])[0],
           'tStat': result[0],
           'p': result[1]}
    stat.append(temp)
stat = pd.DataFrame(stat).set_index('area')
stat.sort_values(by='p', ascending=True,inplace=True)
# Drop NaNs
stat.drop(stat[stat['p'].isna()].index, inplace=True)

_ , corrected = multitest.fdrcorrection(stat['p'], method='indep')
stat['q'] = corrected

print("HFD vs CTR -  WFA diffuse fluo (negative t values mean higher in the HFD group)")
stat.loc[stat['p']<0.05]